In [14]:
'''
pip install tensorflow-addons
pip install tensorflow-hub
pip install tensorflow

'''

'\npip install tensorflow-addons\npip install tensorflow-hub\npip install tensorflow\n\n'

In [1]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

2022-01-19 14:24:18.880533: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-19 14:24:18.880573: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Version:  2.7.0
Eager mode:  True
Hub version:  0.12.0
GPU is NOT AVAILABLE


2022-01-19 14:24:22.216247: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-19 14:24:22.216291: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-19 14:24:22.216309: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (IBS-LAP-314): /proc/driver/nvidia/version does not exist


In [2]:
# Split the training set into 60% and 40% to end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
import os
import pandas as pd
from sklearn.model_selection import train_test_split

train_data, validation_data, test_data = tfds.load(name="imdb_reviews", split=('train[:60%]', 'train[60%:]', 'test'), as_supervised=True)
train_examples_batch, train_labels_batch = next(iter(train_data.batch(5)))
train_examples_batch, train_labels_batch


df = pd.read_json('../dataset/data.json', lines=True)
df = df.replace({'Complaint': {'Yes': 1, 'No': 0}})
X = df['text']
y = df['Complaint']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2


train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_data = tf.data.Dataset.from_tensor_slices((X_test, y_test))
validation_data = tf.data.Dataset.from_tensor_slices((X_val, y_val))


2022-01-19 14:24:22.606450: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [67]:
from sklearn.metrics import fbeta_score, precision_score, recall_score
import tensorflow_addons as tfa
import datetime
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
# embedding = 'https://tfhub.dev/google/Wiki-words-500-with-normalization/2'
embed_dim = 50
lstm_out = 196
max_features=100

hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=False)
# hub_layer(train_examples_batch[:3])
model = tf.keras.Sequential()
model.add(hub_layer)
# model.add(tf.keras.layers.Embedding(max_features, embed_dim, input_length=50))
# model.add(tf.keras.layers.LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

fBetaMetric = tfa.metrics.FBetaScore(num_classes=1, beta=2.0, threshold=0.5)
Precision = tf.keras.metrics.Precision(),
Recall = tf.keras.metrics.Recall()

model.compile(  optimizer='adam', 
                loss='binary_crossentropy',
                metrics=[ Recall, Precision ])
# model.summary()

BATCH_SIZE = 64
SHUFFLE_SIZE = 32

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit(
    train_data.shuffle(SHUFFLE_SIZE).batch(BATCH_SIZE),
    epochs=15,
    validation_data=validation_data.batch(BATCH_SIZE),
    # callbacks=[tensorboard_callback],
    verbose=1)

results = model.evaluate(test_data.batch(BATCH_SIZE), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

y_test_true = [ty.numpy() for (tx, ty) in test_data]
y_test_pred = np.round(model.predict(test_data.batch(BATCH_SIZE)))
fbeta_score(y_test_true, y_test_pred, average='macro', beta=2)


Epoch 1/15
29/29 [==============================] - 1s 8ms/step - loss: 0.5242 - recall_24: 0.1514 - precision_24: 0.1564 - val_loss: 0.3405 - val_recall_24: 0.0000e+00 - val_precision_24: 0.0000e+00
Epoch 2/15
29/29 [==============================] - 0s 3ms/step - loss: 0.3048 - recall_24: 0.0080 - precision_24: 1.0000 - val_loss: 0.2695 - val_recall_24: 0.0120 - val_precision_24: 1.0000
Epoch 3/15
29/29 [==============================] - 0s 3ms/step - loss: 0.2543 - recall_24: 0.1594 - precision_24: 0.6667 - val_loss: 0.2280 - val_recall_24: 0.3012 - val_precision_24: 0.7576
Epoch 4/15
29/29 [==============================] - 0s 3ms/step - loss: 0.2261 - recall_24: 0.4661 - precision_24: 0.7358 - val_loss: 0.2061 - val_recall_24: 0.4458 - val_precision_24: 0.7872
Epoch 5/15
29/29 [==============================] - 0s 3ms/step - loss: 0.1986 - recall_24: 0.5299 - precision_24: 0.7151 - val_loss: 0.1890 - val_recall_24: 0.4578 - val_precision_24: 0.8261
Epoch 6/15
29/29 [==============

0.825897731881571

In [69]:
model.save('../experiments/lstm_frozen')
savedModel = tf.keras.models.load_model('../experiments/lstm_frozen')
# savedModel.predict(test_data.batch(BATCH_SIZE))
y_test_true = [ty.numpy() for (tx, ty) in test_data]
y_test_pred = np.round(savedModel.predict(test_data.batch(BATCH_SIZE)))
fbeta = fbeta_score(y_test_true, y_test_pred, average='macro', beta=2)
prec = precision_score(y_test_true, y_test_pred)
recall = recall_score(y_test_true, y_test_pred)

print(f'Precision: {prec}\tRecall: {recall}\tFbeta: {fbeta}')

INFO:tensorflow:Assets written to: ../experiments/lstm_frozen/assets


INFO:tensorflow:Assets written to: ../experiments/lstm_frozen/assets


Precision: 0.8571428571428571	Recall: 0.6428571428571429	Fbeta: 0.825897731881571
